In [1]:
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import os
import time
import csv
import json
from typing import Dict, List, Optional, Union, cast
import requests
import prepare

from bs4 import BeautifulSoup

df = prepare.wrangle_data()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 391
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             256 non-null    object
 1   language         256 non-null    object
 2   readme_contents  256 non-null    object
 3   stemmed          256 non-null    object
 4   lemmatized       256 non-null    object
dtypes: object(5)
memory usage: 12.0+ KB


In [2]:
df.head()

,repo,language,readme_contents,stemmed,lemmatized
0,dreamingechoes/awesome-mental-health,HTML,"\n<p align=""center""><img src=""./media/logo.png...",aboutsparkl curat list awesom articl websit re...,aboutsparkles curated list awesome article web...
1,Flaque/quirk,TypeScript,🚧🚧🚧\n\n**Quirk is no longer being maintained.*...,quirk longer maintainedquirk start littl thing...,quirk longer maintainedquirk started little th...
4,sepandhaghighi/nafas,Python,"<div align=""center"">\n<img src=""https://github...",nafa &#9; tabl content overview instal usag is...,nafas &#9; table content overview installation...
5,youarerad/youareradweb,TypeScript,"<div align=""center""><img src=""https://res.clou...",welcom rise disord ' websit repow nonprofit co...,welcome rise disorder ' website repowe nonprof...
7,OSMIHelp/osmi-survey-graph,PHP,# 2016 OSMI Survey Graph\n\n## Installing and ...,2016 osmi survey graph instal run copi envexam...,2016 osmi survey graph installing running copy...


In [ ]:
df.language.value_counts()

In [ ]:
df = df[df.language != 'Jupyter Notebook']

In [ ]:
df.info()

In [ ]:
df.info()

In [ ]:
df = df.dropna()
df = df[df['language'].map(df['language'].value_counts()) >= 5]

In [ ]:
df.language.value_counts()

In [ ]:
df = df.dropna()

In [ ]:
df.info()

In [ ]:
df['readme_original'] = df['readme_contents']

In [ ]:
def basic_clean(string):
    string = string.lower()
    string = (unicodedata.normalize('NFKD', string)
                         .encode('ascii', 'ignore')
                         .decode('utf-8', 'ignore')
             )
    string = re.sub(r"[^a-z0-9'\s]", '', string)
    return string

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(basic_clean)

In [ ]:
df.head()

In [ ]:
def clean_html(string):
    string = re.sub(r'<[^>]*>', '', string)
    string = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", '', string)
    string = re.sub(r'\n', '', string)
    string = re.sub(r'\s\s', '', string)
    return string

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(clean_html)

In [ ]:
df.head()

In [ ]:
def tokenize(string):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    return tokenizer.tokenize(string, return_str=True)

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(tokenize)

In [ ]:
def stem(string):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in string.split()]
    return ' '.join(stems)

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(stem)

In [ ]:
df['readme_contents']

In [ ]:
def lemmatize(string):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    return ' '.join(lemmas)

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(lemmatize)

In [ ]:
def remove_stopwords(string, extra_words=[], exclude_words=[]):
    stopword_list = stopwords.words('english')
    
    for word in extra_words:
        stopword_list.append(word)
    
    for word in exclude_words:
        stopword_list.remove(word)
        
    words = string.split()
    filtered_words = [word for word in words if word not in stopword_list]
    return ' '.join(filtered_words)

In [ ]:
df['readme_contents'] = df['readme_contents'].apply(remove_stopwords)

In [ ]:
df['readme_contents']

In [ ]:


def prepare_readme_data(df, column):
    clean_tokens = (df[column].apply(clean_html)
                              .apply(basic_clean)
                              .apply(tokenize)
                              .apply(remove_stopwords)
                   )
    
    for token in clean_tokens:
        token = ' '.join(token).split()
    
    df['stemmed'] = clean_tokens.apply(stem)
    df['lemmatized'] = clean_tokens.apply(lemmatize)
    return df

def wrangle_data():
    data = pd.read_json('data.json')
    return prepare_readme_data(data, 'readme_contents')